# Skill 5: API Search - Make our bot to talk to any API

We have observed the remarkable synergy created by combining **GPT llms with intelligent agents and detailed prompts**. This powerful combination has consistently delivered impressive results. To further capitalize on this capability, we should aim to integrate it with various systems through API communication. Essentially, we can develop within this notebook what is referred to in OpenAI's ChatGPT as 'GPTs.'

Envision a bot that seamlessly integrates with:

- **CRM Systems:** Including Dynamics, Salesforce, and HubSpot.
- **ERP Systems:** Such as SAP, Dynamics, and Oracle.
- **CMS Systems:** Including Adobe, Oracle, and other content management platforms.

The objective is to connect our bot with data repositories, minimizing data duplication as much as possible. These systems typically offer APIs, facilitating programmatic data access.

In this notebook, we plan to develop an agent capable of querying an API to retrieve information and effectively answer questions.
This time we are going to use an open API for currency and digital coins pricing: https://docs.kraken.com/rest/#tag/Market-Data

In [1]:
import os
import json
import requests
from time import sleep
from typing import Dict, List
from pydantic import BaseModel, Extra, root_validator

from langchain_openai import AzureChatOpenAI
from langchain.agents import AgentExecutor, create_openai_tools_agent
from langchain.callbacks.manager import CallbackManager
from langchain.agents import initialize_agent, AgentType
from langchain.tools import BaseTool
from langchain.requests import RequestsWrapper
from langchain.chains import APIChain

from common.callbacks import StdOutCallbackHandler
from common.utils import num_tokens_from_string, reduce_openapi_spec
from common.prompts import APISEARCH_PROMPT

from IPython.display import Markdown, HTML, display  

from dotenv import load_dotenv
load_dotenv("credentials.env")

def printmd(string):
    display(Markdown(string.replace("$","USD ")))


In [2]:
# Set the ENV variables that Langchain needs to connect to Azure OpenAI
os.environ["OPENAI_API_VERSION"] = os.environ["AZURE_OPENAI_API_VERSION"]

In [3]:
cb_handler = StdOutCallbackHandler()
cb_manager = CallbackManager(handlers=[cb_handler])

COMPLETION_TOKENS = 2000


# This notebook needs GPT-4-Turbo (context size of 128k tokens)
llm = AzureChatOpenAI(deployment_name=os.environ["GPT4_DEPLOYMENT_NAME"], 
                      temperature=0.5, max_tokens=COMPLETION_TOKENS, 
                      streaming=True, callback_manager=cb_manager)

## The Logic

By now, you must infer that the solution for an API Agent has to be something like: give the API specification as part of the system prompt to the LLM , then have an agent plan for the right steps to formulate the API call.<br>

Let's do that. But we must first understand the industry standards of Swagger/OpenAPI


## Introduction to OpenAPI (formerly Swagger)

The OpenAPI Specification, previously known as the Swagger Specification, is a specification for a machine-readable interface definition language for describing, producing, consuming and visualizing web services. Previously part of the Swagger framework, it became a separate project in 2016, overseen by the OpenAPI Initiative, an open-source collaboration project of the Linux Foundation.

OpenAPI Specification is an API description format for REST APIs. An OpenAPI file allows you to describe your entire API, including: Available endpoints (/users for example) and operations on each endpoint ( GET /users, POST /users), description, contact information, license, terms of use and other information.

### Let's get the OpenAPI (Swagger) spec from our desired API that we want to talk to
You can also download it from the Kraken website: https://docs.kraken.com/rest/

In [4]:
url = "https://datasetsgptsmartsearch.blob.core.windows.net/apispecs/openapi_kraken.json"
response = requests.get(url + os.environ['BLOB_SAS_TOKEN'])

# Check if the request was successful
if response.status_code == 200:
    spec = response.json()
else:
    spec = None
    print(f"Failed to retrieve data: Status code {response.status_code}")


Let's see how big is this API specification:

In [5]:
# You can check the function "reduce_openapi_spec()" in utils.py
reduced_api_spec = reduce_openapi_spec(spec)

In [6]:
api_tokens = num_tokens_from_string(str(spec))
print("API spec size in tokens:",api_tokens)
api_tokens = num_tokens_from_string(str(reduced_api_spec))
print("Reduced API spec size in tokens:",api_tokens)

API spec size in tokens: 66625
Reduced API spec size in tokens: 57394


Sometimes it makes sense to reduce the size of the API Specs by using the `reduce_openapi_spec` function. It's optional.

#### NOTE: As you can see, a large context LLM is needed. `GPT4-turbo` is necessary for this notebook to run succesfully.

## Question
Let's make a complicated question that requires two distinct API calls to different endpoints:

In [7]:
QUESTION = """
Tell me the price of bitcoin against USD , also the latest OHLC values for Ethereum,
also me also the bid and ask for Euro
"""

## Use a chain to convert the natural language question to an API request using the API specification in the prompt

We can use a nice chain in langchain called APIChain

In [8]:
# Most of APIs require Authorization tokens, so we construct the headers using a lightweight python request wrapper called RequestsWrapper
access_token = "ABCDEFG123456" 
headers = {"Authorization": f"Bearer {access_token}"}
requests_wrapper = RequestsWrapper(headers=headers)

In [9]:
chain = APIChain.from_llm_and_api_docs(
    llm=llm,
    api_docs=str(reduced_api_spec),
    headers=headers,
    verbose=False,
    limit_to_domains=None,
    callback_manager=cb_manager
)


These are the prompts on the APIChain class (one to create the URL endpoint and the other one to use it and get the answer):

In [10]:
chain.api_request_chain.prompt.template

'You are given the below API Documentation:\n{api_docs}\nUsing this documentation, generate the full API url to call for answering the user question.\nYou should build the API url in order to get a response that is as short as possible, while still getting the necessary information to answer the question. Pay attention to deliberately exclude any unnecessary pieces of data in the API call.\n\nQuestion:{question}\nAPI url:'

In [11]:
chain.api_answer_chain.prompt.template

'You are given the below API Documentation:\n{api_docs}\nUsing this documentation, generate the full API url to call for answering the user question.\nYou should build the API url in order to get a response that is as short as possible, while still getting the necessary information to answer the question. Pay attention to deliberately exclude any unnecessary pieces of data in the API call.\n\nQuestion:{question}\nAPI url: {api_url}\n\nHere is the response from the API:\n\n{api_response}\n\nSummarize this response to answer the original question.\n\nSummary:'

In [12]:
try:
    chain.invoke(QUESTION)
except Exception as e:
    response = str(e)

To get the price of Bitcoin against USD, we will use the `Ticker` endpoint. The `pair` parameter should specify `XXBTZUSD` which is the pair for Bitcoin against USD.

For the latest OHLC values for Ethereum, we will use the `OHLC` endpoint. The `pair` parameter should specify `XETHZUSD` which is the pair for Ethereum against USD.

For the bid and ask for Euro, we will use the `Ticker` endpoint again. The `pair` parameter should specify `ZEURZUSD` which is the pair for Euro against USD.

Since we want to make the response as short as possible, we will only include the necessary pairs in the `pair` parameter for each call.

API url for Bitcoin price against USD (Ticker):
```
https://api.kraken.com/0/public/Ticker?pair=XXBTZUSD
```

API url for the latest OHLC values for Ethereum (OHLC):
```
https://api.kraken.com/0/public/OHLC?pair=XETHZUSD&interval=1
```

API url for the bid and ask for Euro (Ticker):
```
https://api.kraken.com/0/public/Ticker?pair=ZEURZUSD
```

Please note that these U

As we have seen before in prior notebooks, a single chain cannot reason/observe/think/retry, so it cannot call multiple endpoints and it doesn't retry or reflect on errors.

## Creating a custom agent that uses the APIChain as a tool

To solve the avobe problem, we can build a REACT Agent that uses the APIChain as a tool to get the information. This agent will create as many calls as needed (using the chain tool) until it answers the question

In [18]:
class MyAPISearch(BaseTool):
    """APIChain as an agent tool"""
    
    name = "apisearch"
    description = "useful when the questions includes the term: apisearch.\n"

    llm: AzureChatOpenAI
    api_spec: str
    headers: dict = {}
    limit_to_domains: list = None
    verbose: bool = False
    
    def _run(self, query: str) -> str:
        
        chain = APIChain.from_llm_and_api_docs(
                            llm=self.llm,
                            api_docs=self.api_spec,
                            headers=self.headers,
                            verbose=self.verbose,
                            limit_to_domains=self.limit_to_domains
                            )
        try:
            sleep(2) # This is optional to avoid possible TPM rate limits
            response = chain.invoke(query)
        except Exception as e:
            response = e
        
        return response
            
    async def _arun(self, query: str) -> str:
        """Use the tool asynchronously."""
        raise NotImplementedError("This Tool does not support async")

Notice below that we are using GPT-35-Turbo-16k (llm_2) for the Tool and GPT-4-turbo (llm_1) for the Agent

In [19]:
tools = [MyAPISearch(llm=llm, api_spec=str(reduced_api_spec), limit_to_domains=None)]
agent = create_openai_tools_agent(llm, tools, APISEARCH_PROMPT)
agent_executor = AgentExecutor(agent=agent, tools=tools, 
                               return_intermediate_steps=True)


In [23]:
%%time 

#As LLMs responses are never the same, we do a for loop in case the answer cannot be parsed according to our prompt instructions
for i in range(2):
    try:
        response = agent_executor.invoke({"question":QUESTION})["output"]
        break
    except Exception as e:
        response = str(e)
        continue
        
printmd(response)

https://api.kraken.com/0/public/Ticker?pair=XBTUSDThe current Bitcoin price in USD is $63,475.90.https://api.kraken.com/0/public/OHLC?pair=ETHUSDThe latest Ethereum OHLC (Open/High/Low/Close) values are as follows:

- Open: $3466.34
- High: $3466.34
- Low: $3466.34
- Close: $3466.34

These values are for the most recent time frame available from the data provided.To get the current bid and ask price for Euro, you would use the Ticker endpoint to retrieve the information for the asset pair that includes Euro. Assuming the asset pair for Euro to USD is "EURUSD", the API URL would be:

```
https://api.kraken.com/0/public/Ticker?pair=EURUSD
``` 

However, if the exact pair code for Euro to USD is different on Kraken, you would need to replace "EURUSD" with the correct pair code used by Kraken.### Bitcoin Price in USD
The current Bitcoin price in USD is **USD 63,475.90**.

### Ethereum OHLC Values
The latest Ethereum OHLC (Open/High/Low/Close) values are:

- **Open**: USD 3466.34
- **High**

### Bitcoin Price in USD
The current Bitcoin price in USD is **USD 63,475.90**.

### Ethereum OHLC Values
The latest Ethereum OHLC (Open/High/Low/Close) values are:

- **Open**: USD 3466.34
- **High**: USD 3466.34
- **Low**: USD 3466.34
- **Close**: USD 3466.34

These values are for the most recent time frame available from the data provided.

### Bid and Ask Price for Euro
Unfortunately, I was unable to retrieve the current bid and ask price for the Euro. However, typically this information can be found using financial APIs or market data services. If you are looking for this information, financial platforms like Kraken provide an API endpoint where you can get the latest bid and ask prices for currency pairs, including the Euro to USD (EURUSD).

CPU times: user 884 ms, sys: 68.6 ms, total: 953 ms
Wall time: 2min 10s


**Great!!** we have now an API Agent using APIChain as a tool, capable of reasoning until it can find the answer.

## Simple APIs

What happens if the API is quite basic, meaning it's just a simple endpoint without a Swagger/OpenAPI definition? Let’s consider the following example:

[CountdownAPI](https://www.countdownapi.com/) is a streamlined version of the eBay API, available as a paid service. We can test it using their demo query, which does not require any Swagger or OpenAPI specification. In this scenario, our main task is to create a tool that retrieves the results. We then pass these results to an agent for analysis, providing answers to user queries, similar to our approach with the Bing Search agent.

An aspect we haven't discussed yet while constructing our API Agent using the APIChain tool is handling situations where either the API specification or the API call results are quite extensive. In such cases, we need to choose between using GPT-4-32k and GPT-4-Turbo.

In the example below, there is no API specification, but the response from the API is rather lengthy. For this scenario, we will employ GPT-4-32k.

In [24]:
# set up the request parameters
params = {
  'api_key': 'demo',
  'type': 'search',
  'ebay_domain': 'ebay.com',
  'search_term': 'memory cards'
}

# make the http GET request to Countdown API
api_result = requests.get('https://api.countdownapi.com/request', params)

num_tokens = num_tokens_from_string(str(api_result.json())) # this is a custom function we created in common/utils.py
print("Token count:",num_tokens,"\n")  

# print the first 2000 characters of JSON response from Countdown API
print(json.dumps(api_result.json())[:2000], "...")

Token count: 14459 

{"request_info": {"success": true, "demo": true}, "request_parameters": {"type": "search", "ebay_domain": "ebay.com", "search_term": "memory cards"}, "request_metadata": {"ebay_url": "https://www.ebay.com/sch/i.html?_nkw=memory+cards&_sacat=0&_dmd=1&_fcid=1"}, "search_results": [{"position": 1, "title": "Sandisk Micro SD Card Memory 32GB 64GB 128GB 256GB 512GB 1TB Lot Extreme Ultra", "epid": "203914554350", "link": "https://www.ebay.com/itm/203914554350", "image": "https://i.ebayimg.com/thumbs/images/g/A7wAAOSwemNjTz~l/s-l300.jpg", "condition": "Brand New", "seller_info": {"name": "terashack", "review_count": 60445, "positive_feedback_percent": 100}, "is_auction": false, "buy_it_now": false, "free_returns": true, "sponsored": true, "prices": [{"value": 9.99, "raw": "$9.99"}, {"value": 438.99, "raw": "$438.99"}], "price": {"value": 9.99, "raw": "$9.99"}}, {"position": 2, "title": "Gigastone 128GB SD Card, Memory Card for Cameras, Full HD Video, SDXC U1 Class10", "ep

So, the answer from this product query (the demo only works with 'memory cards' - you will need to sign up for their trial if you want to try any query with an API key), is about 16.5k tokens. When combined with the prompt, we won't have any other option than to use GPT-4-32k or GPT-4 turbo models. 

In [25]:
class MySimpleAPISearch(BaseTool):
    """Tool for simple API calls that doesn't require OpenAPI 3.0 specs"""
    
    name = "apisearch"
    description = "useful when the questions includes the term: apisearch.\n"

    api_key: str
    
    def _run(self, query: str) -> str:
        
        params = {
          'api_key': self.api_key,
          'type': 'search',
          'ebay_domain': 'ebay.com',
          'search_term': query
        }

        # make the http GET request to Countdown API
        api_result = requests.get('https://api.countdownapi.com/request', params)
        
        try:
            response = json.dumps(api_result.json())
        except Exception as e:
            response = e
        
        return response
            
    async def _arun(self, query: str) -> str:
        """Use the tool asynchronously."""
        raise NotImplementedError("This Tool does not support async")

In [26]:
tools = [MySimpleAPISearch(api_key='demo')]
agent = create_openai_tools_agent(llm, tools, APISEARCH_PROMPT)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=False, return_intermediate_steps=True )

This time let's use the .stream() method 

In [27]:
for chunk in agent_executor.stream({"question": 'what is the price for SanDisk "memory cards"? give me the links please', "language":"English"}):
    # Agent Action
    if "actions" in chunk:
        for action in chunk["actions"]:
            print(f"Calling Tool: `{action.tool}` with input `{action.tool_input}`")
    # Observation
    elif "steps" in chunk:
        continue
        # for step in chunk["steps"]:
        #     print(f"Tool Result: `{step.observation}`")
    # Final result
    elif "output" in chunk:
        printmd(f'Final Output: {chunk["output"]}')
    else:
        raise ValueError()
    print("---")

Calling Tool: `apisearch` with input `SanDisk memory cards price`
---
Here are some options for SanDisk memory cards with their prices and links to purchase them:

1. **Sandisk Micro SD Card Memory**
   - Price: USD 9.99 to USD 438.99
   - [Buy on eBay](https://www.ebay.com/itm/203914554350)

2. **SanDisk Micro SD Card for Smartphones Tablets**
   - Price: USD 5.20 to USD 18.90
   - [Buy on eBay](https://www.ebay.com/itm/324736594273)

3. **Lot of 4 SanDisk Secure Digital Ultra 64GB Memory Card**
   - Price: USD 25.95
   - [Buy on eBay](https://www.ebay.com/itm/386454457401)

4. **SanDisk Industrial Micro SD Memory Card Class 10 UHS-I**
   - Price: USD 9.82 to USD 366.39
   - [Buy on eBay](https://www.ebay.com/itm/274312158070)

5. **Sandisk Micro SD Card Ultra Memory Card Wholesale lot**
   - Price: USD 2.48 to USD 514.20
   - [Buy on eBay](https://www.ebay.com/itm/195635604530)

Please note that prices are subject to change and may vary based on the seller, condition of the card, and

Final Output: Here are some options for SanDisk memory cards with their prices and links to purchase them:

1. **Sandisk Micro SD Card Memory**
   - Price: USD 9.99 to USD 438.99
   - [Buy on eBay](https://www.ebay.com/itm/203914554350)

2. **SanDisk Micro SD Card for Smartphones Tablets**
   - Price: USD 5.20 to USD 18.90
   - [Buy on eBay](https://www.ebay.com/itm/324736594273)

3. **Lot of 4 SanDisk Secure Digital Ultra 64GB Memory Card**
   - Price: USD 25.95
   - [Buy on eBay](https://www.ebay.com/itm/386454457401)

4. **SanDisk Industrial Micro SD Memory Card Class 10 UHS-I**
   - Price: USD 9.82 to USD 366.39
   - [Buy on eBay](https://www.ebay.com/itm/274312158070)

5. **Sandisk Micro SD Card Ultra Memory Card Wholesale lot**
   - Price: USD 2.48 to USD 514.20
   - [Buy on eBay](https://www.ebay.com/itm/195635604530)

Please note that prices are subject to change and may vary based on the seller, condition of the card, and any ongoing promotions or sales. The provided links will take you to eBay where you can view more details and complete your purchase.

---


# Summary

In this notebook, we learned about how to create very smart API agents for simple or complex APIs that use Swagger or OpenAPI specifications.
We see, again, that the key to success is to use: Agents with Expert tools + GPT-4 + good prompts.

As homework, try to create a shopping assistant for Etsy e-commerce site using the following API spec: (you will need to register for free and create an API-Key)

- https://developers.etsy.com/documentation/
- https://www.etsy.com/openapi/generated/oas/3.0.0.json

# NEXT

The Next Notebook will guide you on how we stick everything together. How do we use the features of all notebooks and create a brain agent that can respond to any request accordingly.